In [1]:
from sray.network.mask2former.mask2former import *

In [2]:
import torch
import numpy as np
import os
import glob
import cv2

In [3]:
scannet_dir = 'data/scannet'
img_files = glob.glob(os.path.join(scannet_dir,"*","color","*.jpg"))

In [4]:
img_files[0]

'data/scannet/scene0011_00/color/637.jpg'

In [5]:
def get_img_id_from_img_path(img_path):
    return img_path.split('/')[-1].split('.')[0]

In [13]:
get_scene_fold_from_img_path(img_files[0])

'data/scannet/scene0011_00'

In [6]:
def get_scene_fold_from_img_path(img_path):
    return os.path.join(*(img_path.split('/')[:-2]))

## get_scene_fold_from_img_path(img_files[0])

In [7]:
cfg = {
    'config_file' : '/home/chengshun.wang/pjs/mmsegmentation/m2f_pt/mask2former_r50_scannet_2d_240x320_pretrain.py',
  'checkpoint_file' : '/home/chengshun.wang/pjs/mmsegmentation/m2f_pt/best_mIoU_iter_85000.pth',
}

In [8]:
model = get_m2f(cfg)

Loads checkpoint by local backend from path: /home/chengshun.wang/pjs/mmsegmentation/m2f_pt/best_mIoU_iter_85000.pth


In [9]:
def get_img(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(downsample_gaussian_blur(
        img, 320/1296), (240, 320), interpolation=cv2.INTER_LINEAR)
    return img
def downsample_gaussian_blur(img, ratio):
    sigma = (1 / ratio) / 3
    # ksize=np.ceil(2*sigma)
    ksize = int(np.ceil(((sigma - 0.8) / 0.3 + 1) * 2 + 1))
    ksize = ksize + 1 if ksize % 2 == 0 else ksize
    img = cv2.GaussianBlur(img, (ksize, ksize), sigma, borderType=cv2.BORDER_REFLECT101)
    return img

In [14]:
for i in img_files:
    scene_dir = get_scene_fold_from_img_path(i)
    img_id = get_img_id_from_img_path(i)
    os.makedirs(os.path.join(scene_dir,'m2f'),exist_ok=True)
    seg_logits, pred_sem_seg, mlvl_feats = get_m2f_inference_outputs(model,[get_img(i)])
    seg_logits = seg_logits[0].cpu()
    pred_sem_seg  =pred_sem_seg[0].cpu()
    mlvl_feats = torch.stack(mlvl_feats,1)[0].cpu()
    # print(seg_logits.shape)
    # print(pred_sem_seg.shape)
    # print(mlvl_feats.shape)
    ret=dict(seg_logits=seg_logits, pred_sem_seg = pred_sem_seg, mlvl_feats = mlvl_feats)
    torch.save(ret,os.path.join(scene_dir,'m2f',f'm2f_{img_id}.pt'))
    # # torch.save(ret,os.path.join('m2f.pt'))
    # break

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 44.35 GiB total capacity; 3.12 GiB already allocated; 22.69 MiB free; 3.15 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
data = torch.load('m2f.pt')